In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/algoritmos/Python'

In [4]:
os.chdir(cur_directory + dir_alg_python)

In [5]:
import math

import numpy as np

from numerical_differentiation import gradient_approximation, \
                                      Hessian_approximation
from line_search import line_search_for_residual_by_backtracking
from utils import compute_error


In [6]:
def log_barrier_aux_eval_function(eval_f_const_inequality):
    '''
    Auxiliary function for evaluation of contraint inequalities in logarithmic barrier
    function.
    '''
    #get values that are nonnegative through indexes
    idx_zeros = np.logical_and(eval_f_const_inequality < np.nextafter(0,1),
                               eval_f_const_inequality > -np.nextafter(0,1))
    idx_positive = eval_f_const_inequality > 0 
    idx  = np.logical_or(idx_zeros, idx_positive) 
    #eval constraint inequality functions
    #next line produces warning if a value of constraint is nonnegative
    eval_f_const_inequality = np.log(-eval_f_const_inequality)                                                           
    #assign large value for values positive or equal to 0
    eval_f_const_inequality[idx] = 1e10     
    return eval_f_const_inequality

In [7]:
def constraint_inequalities_funcs_sum_eval(x):
    const_ineq_funcs_eval = np.array([const(x) for const in constraint_inequalities_funcs()])
    return np.sum(log_barrier_aux_eval_function(const_ineq_funcs_eval))

In [8]:
def logarithmic_barrier(x, t_path):
    '''
    Implementation of Logarithmic barrier function.
    '''
    return t_path*f(x) + constraint_inequalities_funcs_sum_eval(x)
           

In [9]:
def numerical_differentiation_of_logarithmic_barrier(x, t_path):
    '''
    First and second derivative of logarithmic barrier function approximation
    via finite differences
    '''
    sum_gf_const = 0
    sum_Hf_const = 0
    for const in constraint_inequalities_funcs():
        const_eval = const(x)
        gf_const_eval = gradient_approximation(const, x)
        Hf_const_eval = Hessian_approximation(const, x)
        print(Hf_const_eval)
        sum_Hf_const = np.outer(gf_const_eval,gf_const_eval)/const_eval**2 - Hf_const_eval/const_eval
        sum_gf_const += gf_const_eval/const_eval  
        
    return [-sum_gf_const,sum_Hf_const]

# Example

In [10]:
f = lambda x: 3*x[0]**2 + 5*x[1]**2

In [11]:
def constraint_inequalities_funcs():
    '''
    Functional form of inequalities.
    For every example this function changes.
    '''
    const = {0: lambda x: 4*x[0] + 5*x[1] -1,
             1: lambda x: 3*x[0]**2 -3*x[1] -10,
             2: lambda x: -x[1],
            }
    for k, v in const.items():
        yield v

In [12]:
x_ex = np.array([1,1],dtype=float)

In [13]:
numerical_differentiation_of_logarithmic_barrier(x_ex,1)

[[-0.00177636 -0.00177636]
 [-0.00177636 -0.00177636]]
[[ 5.99698069e+00 -1.77635684e-03]
 [-1.77635684e-03 -1.77635684e-03]]
[[0.         0.        ]
 [0.         0.00011102]]


[array([ 0.1  , -1.925]), array([[0.        , 0.        ],
        [0.        , 1.00011103]])]

In [14]:
logarithmic_barrier(x_ex,1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


10000000010.302586